MODIFICACIONES LUEGO DE CONCATENAR

In [6]:
#Importamos las librerias a usar
import pandas as pd
from collections import Counter

Cargo los datasets: Amazon, Disney, Hulu y Netflix

In [ ]:
df_A = pd.read_csv("Datasets/amazon_prime_titles.csv")
df_D = pd.read_csv("Datasets/disney_plus_titles.csv")
df_H = pd.read_csv("Datasets/hulu_titles.csv")
df_N = pd.read_csv("Datasets/netflix_titles.csv")

Transformaciones: Propuesta de trabajo (requerimientos de aprobación)

Generar campo id: Cada id se compondrá de la primera letra del nombre de la plataforma, seguido del show_id ya presente en los datasets (ejemplo para títulos de Amazon = as123)

In [ ]:
df_A.insert(0, 'id', 'A' + df_A['show_id'].astype(str))
df_D.insert(0, 'id', 'D' + df_D['show_id'].astype(str))
df_H.insert(0, 'id', 'H' + df_H['show_id'].astype(str))
df_N.insert(0, 'id', 'N' + df_N['show_id'].astype(str))

Unifico los datasets

concatenos los 4 df en uno principal para luego continuar con la transformación de los datos desde un único dataframe y así ahorrarme repetir los mismo pasos

In [ ]:
df = pd.concat([df_D, df_A, df_H, df_N])

Los valores nulos del campo rating deberán reemplazarse por el string “G” (corresponde al maturity rating: “general for all audiences”

In [ ]:
df['rating'].fillna('G', inplace = True)

De haber fechas, deberán tener el formato AAAA-mm-dd

In [ ]:
df['date_added'] = pd.to_datetime(df['date_added'].str.strip(), format='%B %d, %Y')


Los campos de texto deberán estar en minúsculas, sin excepciones

In [ ]:
df = df.applymap(lambda x: x.lower() if type(x) == str else x)

El campo duration debe convertirse en dos campos: duration_int y duration_type. El primero será un integer y el segundo un string indicando la unidad de medición de duración: min (minutos) o season (temporadas)

In [ ]:
df[['duration_int', 'duration_type']] = df['duration'].str.split(expand = True)
df['duration_int'].fillna(0,inplace= True) #esta linea soluciona el error de convertir nulos a enteros, cambio ceros por nulos
df['duration_int'] = df['duration_int'].astype(int)
df.drop(columns = ['duration'], inplace = True)

Cargo los datasets correspondientes a los ratings de los usuarios

In [ ]:
#los puntajes están en otros datasets
rating_1= pd.read_csv('Ratings/1.csv')
rating_2= pd.read_csv('Ratings/2.csv')
rating_3= pd.read_csv('Ratings/3.csv')
rating_4= pd.read_csv('Ratings/4.csv')
rating_5= pd.read_csv('Ratings/5.csv')
rating_6= pd.read_csv('Ratings/6.csv')
rating_7= pd.read_csv('Ratings/7.csv')
rating_8= pd.read_csv('Ratings/8.csv')

In [ ]:
#concateno los df# de los ratings
df_rating = pd.concat([rating_1,rating_2,rating_3,rating_4,rating_5,rating_6,rating_7,rating_8])

Obtengo el promedio de los scores de los dataframes de ratings, lo uno al dataframe principal por el movieId y agrego la columna de los puntajes

In [ ]:
#agrupo por movieId y promedio los puntajes de distintos usuarios
df_grouped = round(df_rating.groupby('movieId', as_index=False)['rating'].mean(),1)
#junto con el dataframe general de plataformas a partir del movieId
df = pd.merge(df , df_grouped, left_on='id',right_on='movieId')
#por último descarto la columna que no me interesa para quedarme únicamente con el promedio de los puntajes
df = df.drop(columns=['movieId'])
#renombro la columna rating_y como score y rating_x como rating
df = df.rename(columns={'rating_y': 'score'})
df = df.rename(columns={'rating_x': 'rating'})


Guardo el dataset limpio en un csv distinto

In [ ]:
df.to_csv('dataset_plataformas.csv', index = False)

In [4]:
df = pd.read_csv('dataset_plataformas.csv')

In [49]:
df.isnull().sum()

id                   0
show_id              0
type                 0
title                0
director          8259
cast              5321
country          11499
date_added        9554
release_year         0
rating               0
listed_in            0
description          4
duration_int         0
duration_type      482
score                0
dtype: int64

In [50]:
df.head()

,id,show_id,type,title,director,cast,country,date_added,release_year,rating,listed_in,description,duration_int,duration_type,score
0,ds1,s1,movie,duck the halls: a mickey mouse christmas special,"alonso ramirez ramos, dave wasson","chris diamantopoulos, tony anselmo, tress macn...",NaN,2021-11-26,2016,tv-g,"animation, family",join mickey and the gang as they duck the halls!,23,min,3.6
1,ds2,s2,movie,ernest saves christmas,john cherry,"jim varney, noelle parker, douglas seale",NaN,2021-11-26,1988,pg,comedy,santa claus passes his magic bag to a new st. ...,91,min,3.5
2,ds3,s3,movie,ice age: a mammoth christmas,karen disher,"raymond albert romano, john leguizamo, denis l...",united states,2021-11-26,2011,tv-g,"animation, comedy, family",sid the sloth is on santa's naughty list.,23,min,3.6
3,ds4,s4,movie,the queen family singalong,hamish hamilton,"darren criss, adam lambert, derek hough, alexa...",NaN,2021-11-26,2021,tv-pg,musical,"this is real life, not just fantasy!",41,min,3.5
4,ds5,s5,tv show,the beatles: get back,NaN,"john lennon, paul mccartney, george harrison, ...",NaN,2021-11-25,2021,g,"docuseries, historical, music",a three-part documentary from peter jackson ca...,1,season,3.5


Desarrollo API: Propones disponibilizar los datos de la empresa usando el framework FastAPI, generando diferentes endpoints que se consumiran en la API.



Creas 6 funciones (recuerda que deben tener un decorador por cada una (@app.get(‘/’)):

1) Película (sólo película, no serie, ni documentales, etc) con mayor duración según año, plataforma y tipo de duración. La función debe llamarse get_max_duration(year, platform, duration_type) y debe devolver sólo el string del nombre de la película.

In [5]:
def get_max_duration(year: int, platform: str, duration_type: str):
    """
    devuelve sólo el string del nombre de la película con mayor duración según año, plataforma y tipo de duración
    """
    #Me aseguro que si el usuario escribe en mayus se pase a minusculas, en el caso de plataform solo me interesa la primer letra.
    platform = platform.lower()[0]
    duration_type = duration_type.lower()
    resultado = df[(df['release_year']==year) & (df['id'].str.startswith(platform)) & (df['duration_type']== duration_type)]
    idx = resultado['duration_int'].idxmax()
    return {'pelicula': resultado.loc[idx, 'title']}

In [7]:
get_max_duration(2018, 'amazon', 'min')

{'pelicula': 'soothing surf at del norte for sleep black screen'}

2) Cantidad de películas (sólo películas, no series, ni documentales, etc) según plataforma, con un puntaje mayor a XX en determinado año. La función debe llamarse get_score_count(platform, scored, year) y debe devolver un int, con el total de películas que cumplen lo solicitado.

In [8]:
def get_score_count(plataforma: str, scored: float, year: int):
    ''' 
    Devuelve un int con el total de películas con puntaje mayor a XX en determinado año
    '''
    #Me aseguro que si el usuario escribe en mayus se pase a minusculas, en el caso de plataform solo me interesa la primer letra.
    platform = plataforma.lower()[0]#Aclaro indice en 0 para que tome la primer letra
    #Filtro por año de lanzamiento, plataforma aclarando que solo tenga en cuenta la primer letra de la columna id y que el tipo sea 'movie'.
    data_filtrada = df[(df['id'].str.startswith(platform)) & (df['release_year'] == year) & (df['type'] == 'movie')]
    #Cuento la cantidad de películas que tienen el rating deseado
    cantidad = data_filtrada[data_filtrada['score'] > scored]['id'].count()
    return {'plataforma': plataforma,
            'cantidad': int(cantidad),
            'anio': year,
            'score': scored}

In [10]:
get_score_count('Hulu',3.2,2018)

{'plataforma': 'Hulu', 'cantidad': 157, 'anio': 2018, 'score': 3.2}

3) Cantidad de películas (sólo películas, no series, ni documentales, etc) según plataforma. La función debe llamarse get_count_platform(platform) y debe devolver un int, con el número total de películas de esa plataforma. Las plataformas deben llamarse amazon, netflix, hulu, disney.

In [25]:
def get_count_platform(platform: str):
    '''
    Devuelve un int con el número total de películas de la plataforma que ingresa
    '''
    #Me aseguro que si el usuario escribe en mayuscula se pase a minusculas, en el caso de plataform solo me interesa la primer letra.
    platform = platform.lower()[0]
    #Realizo el filtro para que cuente del dataframe solo la plataforma de la columna ID y movie de la columna type
    cantidad = df[(df['id'].str.startswith(platform)) & (df['type'] == 'movie')].shape[0]
    return {'plataforma': platform, 'peliculas': cantidad}

In [26]:
get_count_platform('Disney')

{'plataforma': 'd', 'peliculas': 1052}

4. Actor que más se repite según plataforma y año. La función debe llamarse get_actor(platform, year) y debe devolver sólo el string con el nombre del actor que más se repite según la plataforma y el año dado.

In [23]:
def get_actor(platforma: str, year: int):
    ''' 
    Devuelve sólo el  string con el nombre del actor que más se repite según la plataforma y el año dado
    '''
    #Me aseguro que si el usuario escribe en mayus se pase a minusculas, en el caso de plataform solo me interesa la primer letra.
    platform = platforma.lower()[0]
    df_filt = df[(df['id'].str.startswith(platform)) & (df['release_year'] == year)]
    #Utilizo la función dropna() para eliminar los valores nulos de la columna "cast" del DataFrame. 
    #Con el método apply() aplico la función str() a cada valor de la columna "cast", convirtiendo los valores a cadenas. 
    #Uso el método str.split(',') para dividir cada cadena en una lista de actores utilizando la coma como separador. 
    #Obtengo una serie de pandas que contiene una lista de actores para cada fila del DataFrame original.
    actores_por_fila = df_filt['cast'].dropna().apply(lambda x: [i.strip() for i in x.split(',') if not i.strip().isdigit()])
    #Cuento la cantidad de veces que aparece cada actor en todas las filas, utilizando la clase Counter de Python.
    contador_actores = Counter()
    for actores in actores_por_fila:
        contador_actores.update(actores)
    #Encuentro el actor que aparece más veces utilizando la funcion most_common devolviendo una lista de tuplas donde cada tupla contiene un actor 
    #y la cantidad de veces que aparece en todas las filas del DataFrame.
    actor_mas_repetido = contador_actores.most_common(1)
    if actor_mas_repetido:
        #Asigno [0][0] para indicar el actor que mas veces aparece
        actor_mas_repetido = actor_mas_repetido[0][0]
        #Muestro el actor que aparece más veces y la cantidad de veces que aparece
        cantidad_actor_mas_repetido = contador_actores[actor_mas_repetido]
        return {'plataforma': platforma,
                'anio': year,
                'actor': actor_mas_repetido,
                'apariciones': int(cantidad_actor_mas_repetido)}
    else:
        return {'plataforma': platforma,
                'anio': year,
                'actor': "No hay datos disponibles",
                'apariciones': "No hay datos disponibles"}

In [24]:
get_actor('amazon', 2020)

{'plataforma': 'amazon',
 'anio': 2020,
 'actor': 'anushka shetty',
 'apariciones': 7}

5. La cantidad de contenidos/productos (todo lo disponible en streaming) que se publicó por país y año. La función debe llamarse prod_per_county(tipo,pais,anio) deberia devolver el tipo de contenido (pelicula,serie,documental) por pais y año en un diccionario con las variables llamadas 'pais' (nombre del pais), 'anio' (año), 'pelicula' (tipo de contenido).

In [19]:
def prod_per_country(tipo: str, pais: str, year: int):
    ''' 
    Devuelve el tipo de contenido (pelicula,serie,documental) por pais y año en un diccionario con las variables 
    llamadas 'pais' (nombre del pais), 'anio' (año), 'pelicula' (tipo de contenido)
    '''
    #Filtro el dataframe por tipo, país y año
    df_filtrado = df[(df['type'] == tipo) & (df['country'] == pais) & (df['release_year'] == year)]
    #Me quedo con la cantidad de producciones
    cantidad = len(df_filtrado)
    return {'pais': pais,
            'anio': year,
            'pelicuas': int(cantidad)
            }

In [20]:
prod_per_country('movie', 'india', 2020)

{'pais': 'india', 'anio': 2020, 'pelicuas': 83}

6. La cantidad total de contenidos/productos (todo lo disponible en streaming, series, documentales, peliculas, etc) según el rating de audiencia dado (para que publico fue clasificada la pelicula). La función debe llamarse get_contents(rating) y debe devolver el numero total de contenido con ese rating de audiencias.

In [21]:
def get_contents(rating: str):
    ''' 
    Devuelve el numero total de contenido con ese rating de audiencias
    '''
    rating = rating.lower()
    #Filtro el dataframe por tipo, país y año
    df_filtrado = df[df['rating'] == rating]
    #Me quedo con la cantidad de producciones
    cantidad = len(df_filtrado)
    return {'rating': rating, 'contenido': cantidad}


In [22]:
get_contents('r')

{'rating': 'r', 'contenido': 2154}